# Text Processing for Vietnamese Football News

Notebook này xử lý dữ liệu bóng đá từ MongoDB, bao gồm:
- Kết nối MongoDB
- Làm sạch và chuẩn hóa văn bản tiếng Việt
- Tách từ và loại bỏ stopwords
- Trích xuất entities (đội bóng, cầu thủ, giải đấu)
- Phân tích tần suất từ và n-grams

## 1. Import Libraries

In [44]:
import os
import json
import re
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import certifi
from collections import Counter
from tqdm import tqdm
from itertools import islice
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
# Text processing libraries - PyVi
try:
    from pyvi import ViTokenizer, ViPosTagger
    PYVI_AVAILABLE = True
    print("✓ PyVi đã được cài đặt")
except ImportError:
    print("✗ PyVi chưa cài đặt. Cài đặt bằng: pip install pyvi")
    PYVI_AVAILABLE = False

✓ PyVi đã được cài đặt


In [23]:
# Text processing libraries - spaCy
try:
    import spacy
    nlp = spacy.load("vi_core_news_sm")
    SPACY_AVAILABLE = True
    print("✓ spaCy Vietnamese model đã được cài đặt")
except:
    print("✗ spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm")
    SPACY_AVAILABLE = False

✗ spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm


## 2. MongoDB Configuration

In [45]:
# MongoDB connection
MONGO_URI = load_dotenv("MONGO_URI") or os.getenv("MONGO_URI")
DB_NAME = "vnexpress_db"
COLLECTION_NAME = "vnexpress_bongda"

print(f"Database: {DB_NAME}")
print(f"Collection: {COLLECTION_NAME}")

Database: vnexpress_db
Collection: vnexpress_bongda


## 3. Vietnamese Text Processor Class

In [25]:
class VietnameseTextProcessor:
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
        
        # Football-specific entities
        self.football_teams = [
            'hà nội fc', 'hoàng anh gia lai', 'sài gòn fc', 'than quảng ninh', 'viettel fc',
            'becamex bình dương', 'slna', 'đà nẵng', 'nam định', 'hải phòng fc', 'hcm city',
            'song lam nghệ an', 'quảng nam', 'khánh hòa', 'đội tuyển việt nam', 'tuyển việt nam'
        ]
        
        self.player_patterns = [
            r'cầu thủ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền đạo\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'thủ môn\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'hậu vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'HLV\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'huấn luyện viên\s+([A-Za-zÀ-ỹ\s]{3,30})'
        ]
        
        self.competitions = [
            'v-league', 'v.league', 'v league', 'cup quốc gia', 'aff cup', 'sea games',
            'world cup', 'asian cup', 'champions league', 'afc cup', 'u23', 'u22', 'u19'
        ]
    
    def clean_text(self, text):
        """Làm sạch và chuẩn hóa text tiếng Việt"""
        if not text:
            return ""
        
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize_vietnamese(self, text):
        """Tách từ tiếng Việt"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def extract_entities(self, text):
        entities = {'teams': [], 'players': [], 'competitions': []}
        text_lower = text.lower()
        
        for team in self.football_teams:
            if team in text_lower:
                entities['teams'].append(team)
        
        for pattern in self.player_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                player_name = match.group(1).strip()
                if len(player_name) > 3:
                    entities['players'].append(player_name)
        
        for comp in self.competitions:
            if comp in text_lower:
                entities['competitions'].append(comp)
        
        return entities
    
    def process_document(self, doc):
        title = doc.get('title', '')
        content = doc.get('content', '')
        full_text = f"{title} {content}"
        
        cleaned_text = self.clean_text(full_text)
        tokens = self.tokenize_vietnamese(cleaned_text)
        filtered_tokens = self.remove_stopwords(tokens)
        entities = self.extract_entities(full_text)
        
        return {
            'doc_id': str(doc.get('_id', '')),
            'title': title,
            'url': doc.get('url', ''),
            'date': doc.get('date', ''),
            'author': doc.get('author', ''),
            'original_length': len(full_text),
            'cleaned_text': cleaned_text,
            'tokens': tokens,
            'filtered_tokens': filtered_tokens,
            'token_count': len(tokens),
            'filtered_count': len(filtered_tokens),
            'entities': entities,
            'entity_count': sum(len(entities[key]) for key in entities)
        }

print("✓ VietnameseTextProcessor class đã được định nghĩa")

✓ VietnameseTextProcessor class đã được định nghĩa


## 4. Database Functions

In [26]:
def connect_to_database():
    """Kết nối tới MongoDB"""
    try:
        client = MongoClient(MONGO_URI, tls=True, tlsCAFile=certifi.where())
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        doc_count = collection.count_documents({})
        print(f"✓ Kết nối database thành công!")
        print(f"  Tổng số documents: {doc_count}")
        return client, db, collection
    except Exception as e:
        print(f"✗ Lỗi kết nối database: {e}")
        return None, None, None

print("✓ Database functions đã được định nghĩa")

✓ Database functions đã được định nghĩa


## 5. Document Processing Functions

In [27]:
def process_all_documents(collection, limit=None):
    """Xử lý tất cả documents từ collection"""
    processor = VietnameseTextProcessor()
    if limit:
        documents = collection.find().limit(limit)
        total_docs = limit
    else:
        documents = collection.find()
        total_docs = collection.count_documents({})
    
    print(f"Bắt đầu xử lý {total_docs} documents...")
    processed_results, error_count = [], 0
    
    for doc in tqdm(documents, desc="Processing documents", total=total_docs):
        try:
            result = processor.process_document(doc)
            processed_results.append(result)
        except Exception as e:
            error_count += 1
            print(f"Lỗi xử lý document {doc.get('_id', 'unknown')}: {e}")
    
    print(f"✓ Hoàn thành! Xử lý thành công: {len(processed_results)}, Lỗi: {error_count}")
    return processed_results

print("✓ process_all_documents function đã được định nghĩa")

✓ process_all_documents function đã được định nghĩa


## 6. Analysis Functions

In [28]:
def analyze_results(results):
    """Phân tích kết quả xử lý"""
    print("\nPHÂN TÍCH KẾT QUẢ:")
    print(f"Tổng số documents đã xử lý: {len(results)}")
    
    token_counts = [r['token_count'] for r in results]
    filtered_counts = [r['filtered_count'] for r in results]
    entity_counts = [r['entity_count'] for r in results]
    
    print(f"\nThống kê Tokens:")
    print(f"  - Trung bình tokens/document: {sum(token_counts)/len(token_counts):.1f}")
    print(f"  - Trung bình filtered tokens/document: {sum(filtered_counts)/len(filtered_counts):.1f}")
    print(f"  - Trung bình entities/document: {sum(entity_counts)/len(entity_counts):.1f}")
    
    all_teams, all_players, all_competitions = [], [], []
    for r in results:
        all_teams.extend(r['entities']['teams'])
        all_players.extend(r['entities']['players'])
        all_competitions.extend(r['entities']['competitions'])
    
    print(f"\nTop 10 Đội bóng được nhắc đến nhiều nhất:")
    for team, count in Counter(all_teams).most_common(10):
        print(f"  - {team}: {count} lần")
    
    print(f"\nTop 10 Cầu thủ được nhắc đến nhiều nhất:")
    for player, count in Counter(all_players).most_common(10):
        print(f"  - {player}: {count} lần")
    
    print(f"\nTop 10 Giải đấu được nhắc đến nhiều nhất:")
    for comp, count in Counter(all_competitions).most_common(10):
        print(f"  - {comp}: {count} lần")

print("✓ analyze_results function đã được định nghĩa")

✓ analyze_results function đã được định nghĩa


In [29]:
def save_results(results, filename="processed_data.json"):
    """Lưu kết quả ra file JSON"""
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2, default=str)
    print(f"✓ Đã lưu kết quả vào {filepath}")

print("✓ save_results function đã được định nghĩa")

✓ save_results function đã được định nghĩa


## 7. Word Frequency Analysis

In [30]:
def analyze_multiple_documents(collection, processor, n=50, top_n=50, filename=None):
    """Lấy ngẫu nhiên n documents, gộp tokens và tính tần suất chung"""
    docs = collection.aggregate([{"$sample": {"size": n}}])
    all_tokens = []
    
    for idx, doc in enumerate(docs, 1):
        result = processor.process_document(doc)
        tokens = result["filtered_tokens"]
        all_tokens.extend(tokens)   # gộp tất cả tokens lại
    
    # Tính tần suất từ toàn bộ tokens
    counter = Counter(all_tokens)
    total = sum(counter.values())
    
    data = []
    for i, (word, freq) in enumerate(counter.most_common(top_n), 1):
        Pr = freq / total * 100
        data.append({
            "Word": word,
            "Freq": freq,
            "r": i,
            "Pr(%)": round(Pr, 2),
            "r*Pr": round(i * Pr / 100, 3)
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    # Xuất file CSV
    if not filename:
        filename = f"wordfreq_{n}docs_total_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"\n✓ Đã lưu kết quả {n} documents (gộp) vào: {filepath}")
    
    return df

print("✓ analyze_multiple_documents function đã được định nghĩa")

✓ analyze_multiple_documents function đã được định nghĩa


## 8. N-grams Analysis

### Hàm phân tích N-grams

In [31]:
def generate_ngrams(tokens, n):
    """Sinh n-grams từ list tokens"""
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

print("✓ generate_ngrams function đã được định nghĩa")

✓ generate_ngrams function đã được định nghĩa


In [32]:
def analyze_ngrams_combined(collection, processor, n_docs=1800, max_n=5, top_k=50, filename=None):
    """
    Phân tích n-grams (1 -> max_n), lưu tất cả vào 1 file CSV duy nhất
    """
    docs = collection.aggregate([{"$sample": {"size": n_docs}}])

    all_tokens_list = []
    for doc in docs:
        result = processor.process_document(doc)
        all_tokens_list.append(result["filtered_tokens"])

    all_data = []

    for n in range(1, max_n+1):
        ngrams_all = []
        for tokens in all_tokens_list:
            ngrams_all.extend(generate_ngrams(tokens, n))
        
        counter = Counter(ngrams_all)
        for phrase, freq in counter.most_common(top_k):
            all_data.append({
                "n": n,
                "Frequency": freq,
                "Phrase": phrase
            })
    
    df = pd.DataFrame(all_data)

    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    if not filename:
        filename = f"ngrams_combined_{n_docs}docs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"✓ Đã lưu kết quả n-grams (1-{max_n}) vào: {filepath}")
    
    return df

print("✓ analyze_ngrams_combined function đã được định nghĩa")

✓ analyze_ngrams_combined function đã được định nghĩa


In [33]:
def analyze_single_ngram(collection, processor, n=1, n_docs=1800, top_k=100, filename=None):
    """
    Phân tích một loại n-gram cụ thể (1-gram, 2-gram, etc.)
    """
    docs = collection.aggregate([{"$sample": {"size": n_docs}}])

    all_tokens_list = []
    print(f"\nĐang xử lý {n_docs} documents cho {n}-gram...")
    for doc in tqdm(docs, desc=f"Processing {n}-gram", total=n_docs):
        result = processor.process_document(doc)
        all_tokens_list.append(result["filtered_tokens"])

    # Sinh n-grams
    ngrams_all = []
    for tokens in all_tokens_list:
        ngrams_all.extend(generate_ngrams(tokens, n))
    
    # Đếm tần suất
    counter = Counter(ngrams_all)
    total = sum(counter.values())
    
    data = []
    for i, (phrase, freq) in enumerate(counter.most_common(top_k), 1):
        Pr = freq / total * 100
        data.append({
            "Rank": i,
            "Phrase": phrase,
            "Frequency": freq,
            "Pr(%)": round(Pr, 2)
        })
    
    df = pd.DataFrame(data)
    print(f"\n{'='*80}")
    print(f"TOP {top_k} {n}-GRAM")
    print(f"{'='*80}")
    print(df.to_string(index=False))
    
    # Tạo folder outputs nếu chưa có
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    # Lưu file
    if not filename:
        ngram_names = {1: "unigram", 2: "bigram", 3: "trigram", 4: "fourgram", 5: "fivegram"}
        name = ngram_names.get(n, f"{n}gram")
        filename = f"{name}_{n_docs}docs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"\n✓ Đã lưu kết quả {n}-gram vào: {filepath}")
    
    return df

print("✓ analyze_single_ngram function đã được định nghĩa")

✓ analyze_single_ngram function đã được định nghĩa


---

## 8.1. Phân Tích Từng Loại N-gram

### Chạy các cell bên dưới để phân tích từng loại n-gram riêng biệt

### Bước 1: Kết nối Database và khởi tạo Processor

Nếu chưa kết nối, chạy cell này:

In [34]:
# Kết nối database và khởi tạo processor (nếu chưa làm)
try:
    # Kiểm tra xem đã kết nối chưa
    collection.count_documents({})
    print("✓ Đã có kết nối database")
except:
    print("Đang kết nối database...")
    client, db, collection = connect_to_database()
    
try:
    # Kiểm tra processor
    processor
    print("✓ Đã có processor")
except:
    print("Đang khởi tạo processor...")
    processor = VietnameseTextProcessor()
    print("✓ Processor đã sẵn sàng")

Đang kết nối database...
✓ Kết nối database thành công!
  Tổng số documents: 1838
✓ Đã có processor


### Bước 2: Phân tích 1-gram (Unigram)

Phân tích các từ đơn xuất hiện nhiều nhất:

In [35]:
# Phân tích 1-gram (Unigram)
df_unigram = analyze_single_ngram(collection, processor, n=1, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 1-gram...


Processing 1-gram: 100%|██████████| 1830/1830 [00:18<00:00, 98.24it/s] 



TOP 100 1-GRAM
 Rank    Phrase  Frequency  Pr(%)
    1       nam       8284   1.11
    2       đội       7739   1.04
    3      trận       7575   1.02
    4       hai       6793   0.91
    5      việt       6765   0.91
    6   cầu_thủ       6671   0.89
    7      bóng       5690   0.76
    8       hlv       5585   0.75
    9       năm       5201   0.70
   10    league       5152   0.69
   11     trước       4806   0.64
   12       sân       4741   0.64
   13       đến       4510   0.60
   14      phải       4326   0.58
   15       văn       3954   0.53
   16    nguyễn       3943   0.53
   17       bàn       3903   0.52
   18     thắng       3866   0.52
   19     nhiều       3810   0.51
   20      phút       3680   0.49
   21       anh       3651   0.49
   22      nhất       3537   0.47
   23      vòng       3487   0.47
   24       ông       3376   0.45
   25  việt_nam       3360   0.45
   26      giải       3244   0.44
   27        họ       3162   0.42
   28    hà_nội       3129   0.4

### Bước 3: Phân tích 2-gram (Bigram)

Phân tích các cặp từ liên tiếp xuất hiện nhiều nhất:

In [36]:
# Phân tích 2-gram (Bigram)
df_bigram = analyze_single_ngram(collection, processor, n=2, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 2-gram...


Processing 2-gram: 100%|██████████| 1830/1830 [00:18<00:00, 101.24it/s]



TOP 100 2-GRAM
 Rank           Phrase  Frequency  Pr(%)
    1         việt nam       5347   0.72
    2         đội bóng       2052   0.28
    3         thái lan       1888   0.25
    4         nam định       1775   0.24
    5           tp hcm       1336   0.18
    6        hải phòng       1081   0.15
    7       bình dương       1046   0.14
    8        hà_nội fc        991   0.13
    9          chủ nhà        979   0.13
   10        bàn thắng        926   0.12
   11          hlv kim        873   0.12
   12        asean cup        817   0.11
   13         mùa giải        768   0.10
   14        bình định        765   0.10
   15          thứ hai        749   0.10
   16       hiếu lương        718   0.10
   17        quang hải        689   0.09
   18     bóng_đá việt        670   0.09
   19          aff cup        667   0.09
   20         vòng cấm        666   0.09
   21         hiệp hai        654   0.09
   22          hai đội        645   0.09
   23        đội khách        624   0.08


### Bước 4: Phân tích 3-gram (Trigram)

Phân tích các cụm 3 từ liên tiếp xuất hiện nhiều nhất:

In [37]:
# Phân tích 3-gram (Trigram)
df_trigram = analyze_single_ngram(collection, processor, n=3, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 3-gram...


Processing 3-gram: 100%|██████████| 1830/1830 [00:18<00:00, 97.93it/s] 



TOP 100 3-GRAM
 Rank                     Phrase  Frequency  Pr(%)
    1               kim sang sik        620   0.08
    2           bóng_đá việt nam        614   0.08
    3               hlv kim sang        517   0.07
    4             asean cup 2024        473   0.06
    5              park hang seo        355   0.05
    6           cầu_thủ việt nam        348   0.05
    7               sân hàng đẫy        343   0.05
    8               ảnh đức đồng        294   0.04
    9           league 2024 2025        293   0.04
   10                 clb tp hcm        283   0.04
   11             ảnh hiếu lương        276   0.04
   12              hlv park hang        261   0.04
   13              bùi tiến dũng        257   0.03
   14                đội chủ nhà        247   0.03
   15             tuyển việt nam        241   0.03
   16         đội_tuyển việt nam        241   0.03
   17           nguyễn quang hải        236   0.03
   18           nguyễn tiến linh        226   0.03
   19          

### Bước 5: Phân tích 4-gram (Four-gram)

Phân tích các cụm 4 từ liên tiếp xuất hiện nhiều nhất:

In [38]:
# Phân tích 4-gram (Four-gram)
df_fourgram = analyze_single_ngram(collection, processor, n=4, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 4-gram...


Processing 4-gram: 100%|██████████| 1830/1830 [00:18<00:00, 97.55it/s] 



TOP 100 4-GRAM
 Rank                                Phrase  Frequency  Pr(%)
    1                      hlv kim sang sik        515   0.07
    2                     hlv park hang seo        259   0.03
    3              vòng_loại cuối asian cup        141   0.02
    4                    bùi hoàng việt anh        128   0.02
    5                   cuối asian cup 2027        121   0.02
    6                      đội bóng phố núi        112   0.02
    7                      hlv vũ hồng việt        105   0.01
    8                   hlv chu đình nghiêm        103   0.01
    9          vòng_loại hai world_cup 2026        102   0.01
   10                     hlv vũ tiến thành         97   0.01
   11        bóng_đá chuyên_nghiệp việt nam         92   0.01
   12                     mang dòng máu lai         89   0.01
   13                  league 2024 2025 sân         78   0.01
   14                      việt trì phú thọ         77   0.01
   15            liên_đoàn bóng_đá việt nam         77

### Bước 6: Phân tích 5-gram (Five-gram)

Phân tích các cụm 5 từ liên tiếp xuất hiện nhiều nhất:

In [39]:
# Phân tích 5-gram (Five-gram)
df_fivegram = analyze_single_ngram(collection, processor, n=5, n_docs=1830, top_k=100)


Đang xử lý 1830 documents cho 5-gram...


Processing 5-gram: 100%|██████████| 1830/1830 [00:18<00:00, 98.45it/s] 



TOP 100 5-GRAM
 Rank                                     Phrase  Frequency  Pr(%)
    1              vòng_loại cuối asian cup 2027        121   0.02
    2 công_ty cổ_phần bóng_đá chuyên_nghiệp việt         72   0.01
    3     cổ_phần bóng_đá chuyên_nghiệp việt nam         72   0.01
    4              bảng vòng_loại cuối asian cup         72   0.01
    5         bóng_đá chuyên_nghiệp việt nam vpf         69   0.01
    6                       sân việt trì phú thọ         67   0.01
    7                  cầu_thủ mang dòng máu lai         64   0.01
    8                      aff cup nay asean cup         47   0.01
    9             liên_đoàn bóng_đá việt nam vff         43   0.01
   10                     thời hlv park hang seo         39   0.01
   11                  night wolf league 2023 24         35   0.00
   12               lượt hai bảng vòng_loại cuối         35   0.00
   13              hai bảng vòng_loại cuối asian         35   0.00
   14                      thầy trò hlv kim sa

### Bước 7: Tổng hợp tất cả N-grams (Optional)

Nếu muốn lưu tất cả n-grams vào 1 file duy nhất:

In [40]:
# Phân tích và lưu tất cả n-grams vào 1 file
df_all_ngrams = analyze_ngrams_combined(collection, processor, n_docs=1830, max_n=5, top_k=100)

✓ Đã lưu kết quả n-grams (1-5) vào: d:\data\Search_Engine\outputs\ngrams_combined_1830docs_20251006_220331.csv


## 9. Main Execution

### Chạy cell bên dưới để thực hiện toàn bộ quy trình xử lý

In [41]:
def main():
    print("BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB")
    print("=" * 50)
    
    client, db, collection = connect_to_database()
    if collection is None:
        print("Không thể kết nối database!")
        return
    
    # Xử lý một số documents mẫu
    results = process_all_documents(collection, limit=5)  
    if results:
        analyze_results(results)
        save_results(results, f"processed_vnexpress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    
    # Phân tích từ vựng
    processor = VietnameseTextProcessor()
    analyze_multiple_documents(collection, processor, n=50, top_n=50)
    
    # Phân tích n-grams
    analyze_and_save_ngrams(collection, processor, n_docs=1830, max_n=5, top_k=100)

    client.close()
    print("\n✓ Đã đóng kết nối database")
    print("✓ HOÀN THÀNH!")

print("✓ main function đã được định nghĩa")

✓ main function đã được định nghĩa


### Chạy toàn bộ quy trình

Chạy cell bên dưới để thực thi:

In [42]:
def main():
    print("BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB")
    print("=" * 50)
    
    # Tạo folder outputs
    output_dir = os.path.join("d:\\data\\Search_Engine", "outputs")
    os.makedirs(output_dir, exist_ok=True)
    print(f"✓ Folder outputs: {output_dir}\n")
    
    client, db, collection = connect_to_database()
    if collection is None:
        print("Không thể kết nối database!")
        return
    
    # Xử lý một số documents mẫu
    results = process_all_documents(collection, limit=5)  
    if results:
        analyze_results(results)
        save_results(results, f"processed_vnexpress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    
    # Phân tích từ vựng
    processor = VietnameseTextProcessor()
    analyze_multiple_documents(collection, processor, n=50, top_n=50)
    
    # Phân tích từng loại n-gram riêng biệt
    print("\n" + "="*80)
    print("BẮT ĐẦU PHÂN TÍCH N-GRAMS")
    print("="*80)
    
    for n in range(1, 6):
        analyze_single_ngram(collection, processor, n=n, n_docs=1830, top_k=100)
        print("\n")

    client.close()
    print("\n✓ Đã đóng kết nối database")
    print("✓ HOÀN THÀNH!")
    print(f"\n📁 Tất cả kết quả đã được lưu vào folder: {output_dir}")

print("✓ main function đã được định nghĩa")

# Chạy toàn bộ quy trình xử lý
if __name__ == "__main__":
    main()

✓ main function đã được định nghĩa
BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB
✓ Folder outputs: d:\data\Search_Engine\outputs

✓ Kết nối database thành công!
  Tổng số documents: 1838
Bắt đầu xử lý 5 documents...
✓ Kết nối database thành công!
  Tổng số documents: 1838
Bắt đầu xử lý 5 documents...


Processing documents: 100%|██████████| 5/5 [00:00<00:00, 19.46it/s]



✓ Hoàn thành! Xử lý thành công: 5, Lỗi: 0

PHÂN TÍCH KẾT QUẢ:
Tổng số documents đã xử lý: 5

Thống kê Tokens:
  - Trung bình tokens/document: 727.4
  - Trung bình filtered tokens/document: 541.4
  - Trung bình entities/document: 19.2

Top 10 Đội bóng được nhắc đến nhiều nhất:
  - nam định: 2 lần
  - tuyển việt nam: 2 lần
  - hà nội fc: 1 lần
  - đà nẵng: 1 lần
  - đội tuyển việt nam: 1 lần

Top 10 Cầu thủ được nhắc đến nhiều nhất:
  - Kim Sang: 3 lần
  - Park Hang: 2 lần
  - trẻ cần được thi đấu thường xu: 1 lần
  - nhập tịch vì sử dụng giấy tờ g: 1 lần
  - vào sân: 1 lần
  - bị phạt: 1 lần
  - của Quy định này có đoạn: 1 lần
  - phải được đăng ký đúng quy địn: 1 lần
  - được coi là đủ tư cách thi đấu: 1 lần
  - hợp lệ: 1 lần

Top 10 Giải đấu được nhắc đến nhiều nhất:
  - u23: 4 lần
  - world cup: 3 lần
  - asian cup: 3 lần
  - v-league: 2 lần
  - champions league: 1 lần
  - aff cup: 1 lần
✓ Đã lưu kết quả vào d:\data\Search_Engine\outputs\processed_vnexpress_20251006_220337.json
    W

Processing 1-gram: 100%|██████████| 1830/1830 [00:18<00:00, 96.71it/s] 




TOP 100 1-GRAM
 Rank    Phrase  Frequency  Pr(%)
    1       nam       8266   1.11
    2       đội       7743   1.04
    3      trận       7589   1.02
    4       hai       6783   0.91
    5      việt       6763   0.91
    6   cầu_thủ       6667   0.89
    7      bóng       5707   0.77
    8       hlv       5611   0.75
    9       năm       5189   0.70
   10    league       5174   0.69
   11     trước       4808   0.64
   12       sân       4743   0.64
   13       đến       4512   0.61
   14      phải       4329   0.58
   15       văn       3954   0.53
   16    nguyễn       3930   0.53
   17       bàn       3919   0.53
   18     thắng       3867   0.52
   19     nhiều       3814   0.51
   20      phút       3686   0.49
   21       anh       3638   0.49
   22      nhất       3534   0.47
   23      vòng       3491   0.47
   24       ông       3374   0.45
   25  việt_nam       3354   0.45
   26      giải       3241   0.43
   27        họ       3164   0.42
   28    hà_nội       3130   0.4

...


TOP 100 3-GRAM
 Rank                     Phrase  Frequency  Pr(%)
    1               kim sang sik        620   0.08
    2           bóng_đá việt nam        614   0.08
    3               hlv kim sang        517   0.07
    4             asean cup 2024        475   0.06
    5              park hang seo        354   0.05
    6               sân hàng đẫy        343   0.05
    7           cầu_thủ việt nam        338   0.05
    8               ảnh đức đồng        295   0.04
    9           league 2024 2025        291   0.04
   10                 clb tp hcm        279   0.04
   11             ảnh hiếu lương        275   0.04
   12              hlv park hang        262   0.04
   13              bùi tiến dũng        258   0.03
   14                đội chủ nhà        250   0.03
   15         đội_tuyển việt nam        242   0.03
   16             tuyển việt nam        241   0.03
   17           nguyễn quang hải        234   0.03
   18           nguyễn tiến linh        226   0.03
   19          

Processing 4-gram: 100%|██████████| 1830/1830 [00:17<00:00, 102.62it/s]




TOP 100 4-GRAM
 Rank                                Phrase  Frequency  Pr(%)
    1                      hlv kim sang sik        512   0.07
    2                     hlv park hang seo        261   0.04
    3              vòng_loại cuối asian cup        141   0.02
    4                    bùi hoàng việt anh        128   0.02
    5                   cuối asian cup 2027        121   0.02
    6                      đội bóng phố núi        112   0.02
    7                      hlv vũ hồng việt        105   0.01
    8                   hlv chu đình nghiêm        103   0.01
    9          vòng_loại hai world_cup 2026        101   0.01
   10                     hlv vũ tiến thành         97   0.01
   11        bóng_đá chuyên_nghiệp việt nam         91   0.01
   12                     mang dòng máu lai         89   0.01
   13                      việt trì phú thọ         78   0.01
   14                  league 2024 2025 sân         78   0.01
   15            liên_đoàn bóng_đá việt nam         76

Processing 5-gram: 100%|██████████| 1830/1830 [00:18<00:00, 96.66it/s] 




TOP 100 5-GRAM
 Rank                                     Phrase  Frequency  Pr(%)
    1              vòng_loại cuối asian cup 2027        120   0.02
    2 công_ty cổ_phần bóng_đá chuyên_nghiệp việt         72   0.01
    3     cổ_phần bóng_đá chuyên_nghiệp việt nam         72   0.01
    4              bảng vòng_loại cuối asian cup         71   0.01
    5         bóng_đá chuyên_nghiệp việt nam vpf         69   0.01
    6                       sân việt trì phú thọ         67   0.01
    7                  cầu_thủ mang dòng máu lai         63   0.01
    8                      aff cup nay asean cup         47   0.01
    9             liên_đoàn bóng_đá việt nam vff         43   0.01
   10                     thời hlv park hang seo         39   0.01
   11                  night wolf league 2023 24         35   0.00
   12                      thầy trò hlv kim sang         35   0.00
   13                       trò hlv kim sang sik         35   0.00
   14               lượt hai bảng vòng_loại cu